In [602]:
import numpy as np
import pandas as pd
df = pd.read_csv('/Users/avinash.pathak/kaggle/train.csv')

#80-20 split
train_df=df.sample(frac=0.8,random_state=200)
test=df.drop(train_df.index)

df = train_df

In [603]:
X=df.drop('label',axis=1)
y=df['label']

In [604]:
df_train=df.copy()

In [605]:
df.isnull().sum()

id           0
title      439
author    1561
text        32
label        0
dtype: int64

In [606]:
test.isnull().sum()

id          0
title     119
author    396
text        7
label       0
dtype: int64

In [607]:
#as this is a text processing, replacing null values with blanks
df=df.fillna(' ')
test=test.fillna(' ')


In [608]:
# In my analysis I have used title+author+text but title+author works best for this use case in particular.
# So going with that
#df['total'] = df['title']+' '+df['author']+' '+df['text']
#test['total']=test['title']+' '+test['author']+' '+test['text']

df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']


In [609]:
X = df.drop('label',axis=1)
y=df['label']
print(X.shape)
print(y.shape)


(16640, 5)
(16640,)


In [610]:
#some preliminary pre-processing
import nltk
import re
from nltk.corpus import stopwords
# this is a one-time download
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/avinash.pathak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/avinash.pathak/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/avinash.pathak/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [611]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps=PorterStemmer()
wordnet = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [612]:
def proprocess(text):
    #convert all the text to lowercase
    text = text.lower()
    #remove non-ascii chars
    text = text.encode("ascii", "ignore").decode()
    #remove new-line chars
    text = re.sub(r'\n',' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'[0-9]', ' ', text)
    text=text.split()
    return ' '.join([wordnet.lemmatize(word) for word in text if word not in stop_words])

In [613]:
df['total']=df['total'].apply(proprocess)

In [614]:
X=df['total']
y=df['label']

In [615]:
test['total']=test['total'].apply(proprocess)

In [616]:
test_X=test['total']
test_y=test['label']

In [617]:
# random voc size
voc_size=5000

In [618]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_rep = [one_hot(words,voc_size)for words in X]
onehot_rep_test = [one_hot(words,voc_size)for words in test_X]

In [619]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

### Creating and training model

In [620]:
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout, GRU, SimpleRNN
from tensorflow.keras.models import Sequential

In [621]:
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
print(model.summary())

Model: "sequential_65"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 25, 40)            200000    
_________________________________________________________________
dropout_103 (Dropout)        (None, 25, 40)            0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 100)               56400     
_________________________________________________________________
dropout_104 (Dropout)        (None, 100)               0         
_________________________________________________________________
dense_129 (Dense)            (None, 64)                6464      
_________________________________________________________________
dropout_105 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_130 (Dense)            (None, 1)               

In [622]:
#need to conver to numpy arrays
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
y_test_label = np.array(test_y)

In [623]:
X_final.shape,y_final.shape,test_final.shape,y_test_label.shape

((16640, 25), (16640,), (4160, 25), (4160,))

In [624]:
X_final.shape

(16640, 25)

In [625]:
#fit
model.fit(X_final,y_final,epochs=20,batch_size=64)


Epoch 1/20
260/260 [==============================] - 4s 17ms/step - loss: 0.1697 - accuracy: 0.9234 - precision_50: 0.8972 - recall_50: 0.9552
Epoch 2/20
260/260 [==============================] - 4s 15ms/step - loss: 0.0198 - accuracy: 0.9947 - precision_50: 0.9947 - recall_50: 0.9946
Epoch 3/20
260/260 [==============================] - 4s 16ms/step - loss: 0.0089 - accuracy: 0.9977 - precision_50: 0.9979 - recall_50: 0.9975
Epoch 4/20
260/260 [==============================] - 4s 16ms/step - loss: 0.0046 - accuracy: 0.9990 - precision_50: 0.9988 - recall_50: 0.9992
Epoch 5/20
260/260 [==============================] - 4s 14ms/step - loss: 0.0030 - accuracy: 0.9992 - precision_50: 0.9993 - recall_50: 0.9990
Epoch 6/20
260/260 [==============================] - 4s 14ms/step - loss: 0.0022 - accuracy: 0.9993 - precision_50: 0.9994 - recall_50: 0.9993
Epoch 7/20
260/260 [==============================] - 4s 14ms/step - loss: 0.0015 - accuracy: 0.9996 - precision_50: 0.9998 - recall_50:

# Evaluating the model with 20% test data from train set itself

In [626]:
test_final.shape,y_test_label.shape

((4160, 25), (4160,))

In [627]:
score, accuracy, precision, recall = model.evaluate(test_final, y_test_label,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', accuracy)
print('Test precision:', precision)
print('Test recall:', recall)


65/65 [==============================] - 1s 9ms/step - loss: 0.0743 - accuracy: 0.9875 - precision_50: 0.9852 - recall_50: 0.9907
Test score: 0.07431173324584961
Test accuracy: 0.987500011920929
Test precision: 0.9851989150047302
Test recall: 0.9906976819038391


# Calculating confusion matrix for manual precision, recall calculation

In [628]:
y_pred = model.predict(test_final)

In [629]:
y_pred.shape

(4160, 1)

In [630]:
y_pred=np.round(y_pred)

In [631]:
y_test_label.shape,y_pred.shape

((4160,), (4160, 1))

In [632]:
from sklearn.metrics import classification_report, confusion_matrix


print('Confusion Matrix')
conf_matrix = confusion_matrix(y_test_label, y_pred)
print(conf_matrix)

print(conf_matrix.shape)


Confusion Matrix
[[1978   32]
 [  20 2130]]
(2, 2)


### Calculate precision and recall

In [633]:
# for one run these were the values
# tp=1999
# fp=11
# fn=37
# tn=2113

tp = conf_matrix[0][0]
fp = conf_matrix[0][1]
fn = conf_matrix[1][0]
tn = conf_matrix[1][1]

precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("precision = ", precision)
print("recall = ", recall)

precision =  0.9840796019900497
recall =  0.98998998998999


# Following is for submissions.csv, this is done on separate test set provided

In [634]:
test = pd.read_csv('/Users/avinash.pathak/kaggle/test.csv')

In [635]:
test=test.fillna('')

In [636]:
test['total']=test['title']+' '+test['author']

In [637]:
test['total']=test['total'].apply(proprocess)

In [638]:
test_X=test['total']

In [639]:
onehot_rep_test = [one_hot(words,voc_size)for words in test_X]

In [640]:
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [641]:
test_final = np.array(embedded_docs_test)

In [642]:
test_final.shape

(5200, 25)

In [643]:
y_pred = model.predict(test_final)

In [644]:
y_pred=np.round(y_pred)

In [645]:
y_pred.astype('int64')

array([[0],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [646]:
final_sub = pd.DataFrame()
final_sub['id']=test['id']
final_sub['label'] = y_pred
final_sub.to_csv('final_sub.csv',index=False)

In [647]:
final_sub.head()

,id,label
0,20800,0.0
1,20801,1.0
2,20802,1.0
3,20803,0.0
4,20804,1.0


In [648]:
final_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      5200 non-null   int64  
 1   label   5200 non-null   float32
dtypes: float32(1), int64(1)
memory usage: 61.1 KB


In [649]:
final_sub['label']=final_sub['label'].astype('int64')

In [650]:
final_sub.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [651]:
final_sub.to_csv('tomtom_submission.csv',index=False)